<a href="https://colab.research.google.com/github/renz9000/work17-10-68/blob/main/%E0%B8%87%E0%B8%B2%E0%B8%99%E0%B8%AA%E0%B8%96%E0%B8%B4%E0%B8%95%E0%B8%B4V_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import os
from collections import Counter

def clear_screen():
    """ล้างหน้าจอ (ใช้ได้ทั้ง Windows และ macOS/Linux)"""
    os.system('cls' if os.name == 'nt' else 'clear')

def mean(data):
    return sum(data)/len(data) if len(data) > 0 else 0

def median(data):
    if not data: return 0
    sorted_data = sorted(data)
    n = len(sorted_data)
    if n % 2 == 0:
        return (sorted_data[n//2 - 1] + sorted_data[n//2]) / 2
    else:
        return sorted_data[n//2]

def mode(data):
    if not data: return None
    freq = Counter(data)
    max_freq = max(freq.values())
    modes = [k for k, v in freq.items() if v == max_freq]
    if len(modes) == len(data):
        return None  # ไม่มีฐานนิยม
    return modes

def mean_deviation(data):
    if len(data) == 0: return 0
    x_bar = mean(data)
    # สูตร M.D. = sum(|X - X_bar|) / N
    return sum(abs(x - x_bar) for x in data) / len(data)

def variance(data):
    if len(data) <= 1: return 0
    x_bar = mean(data)
    # สูตร S^2 = sum((X - X_bar)^2) / (N-1)
    return sum((x - x_bar)**2 for x in data) / (len(data)-1)

def std_dev(data):
    # สูตร S.D. = sqrt(sum((X - X_bar)^2) / (N-1))
    return math.sqrt(variance(data))

# ---------------------------
# ส่วนที่ 2: แจกแจงความถี่
# ---------------------------

def grouped_statistics(data, num_classes):
    if len(data) == 0 or num_classes <= 0:
        return {}

    data.sort()
    min_val, max_val = min(data), max(data)
    R = max_val - min_val
    i = math.ceil((R + 1) / num_classes) if num_classes != 0 else 1

    # สร้างช่วงชั้น
    classes = []
    lower = min_val
    for _ in range(num_classes):
        upper = lower + i - 1
        classes.append((lower, upper))
        lower = upper + 1

    # ความถี่ (f)
    freq = [sum(1 for x in data if lo <= x <= hi) for lo, hi in classes]
    total_f = sum(freq)

    # ความถี่สะสม (F)
    cum_freq = []
    running = 0
    for f in freq:
        running += f
        cum_freq.append(running)

    # จุดกึ่งกลาง (x)
    mid = [(lo + hi)/2 for lo, hi in classes]

    # fx, สัดส่วน, ร้อยละ
    fx = [mid[i] * freq[i] for i in range(num_classes)]
    prop = [f / total_f for f in freq]
    perc = [p * 100 for p in prop]

    # ใช้สูตร S^2 = [n * sum(fx^2) - (sum(fx))^2] / [n * (n-1)]

    n = total_f
    sum_fx = sum(fx)
    fx2 = [freq[i] * (mid[i]**2) for i in range(num_classes)]
    sum_fx2 = sum(fx2)

    if n > 1:
        numerator = (n * sum_fx2) - (sum_fx ** 2)
        denominator = n * (n - 1)

        # ป้องกันการหารด้วย 0 ถ้า n=1 (แม้ว่าเราจะเช็ค n > 1 แล้วก็ตาม)
        variance_g = numerator / denominator if denominator != 0 else 0

        # ป้องกัน math domain error ถ้า variance เป็นค่าลบเล็กน้อยจาก floating point precision
        sd_g = math.sqrt(variance_g) if variance_g >= 0 else 0
    else:
        variance_g = 0
        sd_g = 0


    # ค่าเฉลี่ย
    mean_g = sum_fx / n if n > 0 else 0


    # มัธยฐาน (Md)
    N = total_f
    median_pos = N / 2
    Md = None
    for j, F in enumerate(cum_freq):
        if F >= median_pos:
            Lm = classes[j][0] - 0.5 # L (ขีดจำกัดล่างที่แท้จริง)
            F_prev = cum_freq[j - 1] if j > 0 else 0 # F (ความถี่สะสมก่อนหน้า)
            fm = freq[j] # f (ความถี่)
            Md = Lm + ((median_pos - F_prev) / fm) * i if fm != 0 else None
            break

    # ฐานนิยม (Mo)
    Mo = None
    if freq: # ตรวจสอบว่ามีข้อมูลความถี่หรือไม่
        max_f = max(freq)
        if max_f > 0:
            modal_index = freq.index(max_f)
            L = classes[modal_index][0] - 0.5
            d1 = freq[modal_index] - (freq[modal_index-1] if modal_index > 0 else 0)
            d2 = freq[modal_index] - (freq[modal_index+1] if modal_index < num_classes-1 else 0)
            Mo = L + (d1 / (d1 + d2)) * i if (d1 + d2) != 0 else L

    # คำนวณควอไทล์ Q1, Q3 และ Q.D.
    def find_quartile(x): # x คือตำแหน่งที่ของควอไทล์ (1, 2, 3)
        pos = (N * x) / 4
        for j, F in enumerate(cum_freq):
            if F >= pos:
                Lq = classes[j][0] - 0.5 # Lo
                F_prev = cum_freq[j-1] if j > 0 else 0 # F
                fq = freq[j] # f
                if fq == 0: return None # ป้องกันการหารด้วย 0
                return Lq + ((pos - F_prev) / fq) * i
        return None

    Q1 = find_quartile(1)
    Q3 = find_quartile(3)
    QD = (Q3 - Q1) / 2 if Q1 is not None and Q3 is not None else None

    return {
        "classes": classes,
        "freq": freq,
        "cum_freq": cum_freq,
        "mid": mid,
        "fx": fx,
        "prop": prop,
        "perc": perc,
        "mean": mean_g,
        "variance": variance_g,
        "sd": sd_g,
        "Mo": Mo,
        "Md": Md,
        "Q1": Q1,
        "Q3": Q3,
        "QD": QD,
        "total_f": total_f
    }

# เมนูหลัก

def main():
    data = []  # เก็บข้อมูลล่าสุดไว้
    while True:
        clear_screen()
        print("=== โปรแกรมคำนวณค่าสถิติ ===")
        print("\n1. ข้อมูลดิบ (ไม่แจกแจงความถี่)")
        print("2. ข้อมูลแจกแจงความถี่")
        print("0. ออกจากโปรแกรม")
        choice = input("\nเลือกโหมด (0, 1 หรือ 2): ")

        if choice == "0":
            clear_screen()
            print("ขอบคุณที่ใช้โปรแกรม")
            break

        # ถามว่าต้องการใช้ข้อมูลเดิมหรือไม่
        if data:
            reuse = input("ใช้ข้อมูลเดิมหรือไม่ (y/n): ").lower()
        else:
            reuse = "n"

        if reuse != "y":
            data = []
            print("\nป้อนข้อมูล (1-999) พิมพ์ 0 เพื่อหยุด:")
            while len(data) < 100: # จำกัดสูงสุด 100 ค่า
                try:
                    x_input = input(f"({len(data)+1})> ")
                    x = int(x_input)
                    if x == 0: break
                    if 1 <= x <= 999: # เฉพาะค่าบวก 1-999
                        data.append(x)
                    else:
                        print("กรุณาป้อนค่าระหว่าง 1 - 999 หรือ 0 เพื่อหยุด")
                except:
                    print("กรุณาป้อนตัวเลขเท่านั้น")

            if len(data) >= 100:
                print("รับข้อมูลครบ 100 ค่าแล้ว")

        if len(data) == 0:
            print("ไม่มีข้อมูล")
            input("กด Enter เพื่อกลับเมนูหลัก...")
            continue

        clear_screen()

        if choice == "1":
            print("--- ผลลัพธ์ข้อมูลดิบ ---")
            print(f"จำนวนข้อมูล = {len(data)}")
            print(f"ค่าสูงสุด(Max) = {max(data)}")
            print(f"ค่าต่ำสุด(Min) = {min(data)}")
            print(f"ค่ามัชฌิมเลขคณิต(Mean) = {round(mean(data), 3)}")
            print(f"ค่ามัธยฐาน(Median) = {round(median(data), 3)}")
            print(f"ค่าฐานนิยม(Mode) = {mode(data)}")
            print(f"ค่าความเบี่ยงเบนเฉลี่ย(Mean Deviation) = {round(mean_deviation(data), 3)}")
            print(f"ค่าความแปรปรวน(Variance) = {round(variance(data), 3)}")
            print(f"ค่าความเบี่ยงเบนมาตรฐาน(Std Deviation) = {round(std_dev(data), 3)}")

        elif choice == "2":

            num_classes = 0
            while True:
                try:
                    num_classes_input = input("ป้อนจำนวนชั้น (ต้องอยู่ระหว่าง 3 - 15): ")
                    num_classes = int(num_classes_input)
                    if 3 <= num_classes <= 15: #
                        break
                    else:
                        print("Error: จำนวนชั้นต้องอยู่ระหว่าง 3 ถึง 15 เท่านั้น ")
                except ValueError:
                    print("Error: กรุณาป้อนเป็นตัวเลข")

            result = grouped_statistics(data, num_classes)
            if not result:
                print("ข้อมูลไม่เพียงพอหรือจำนวนชั้นไม่ถูกต้อง")
                input("กด Enter เพื่อกลับเมนูหลัก...")
                continue

            print("\n=== ตารางแจกแจงความถี่ ===")
            print("ช่วงคะแนน | Lo | Ho | จุดกลาง(x) | f | F | fx | สัดส่วน | ร้อยละ")
            print("-" * 75)
            for i in range(len(result["classes"])):
                lo, hi = result["classes"][i]
                print(f"{lo:>3}-{hi:<3} | {lo:>3} | {hi:>3} | {result['mid'][i]:>8.2f} | "
                      f"{result['freq'][i]:>3} | {result['cum_freq'][i]:>3} | {result['fx'][i]:>7.2f} | "
                      f"{result['prop'][i]:>7.3f} | {result['perc'][i]:>6.2f}")

            print("-" * 75)
            print(f"รวมทั้งหมด: {result['total_f']} | fx รวม = {sum(result['fx']):.2f}")

            print("\n--- ค่าสถิติ  ---")
            print(f"ค่าความเบี่ยงเบนควอไทล์ (Q.D.) = {result['QD']:.3f}" if result['QD'] is not None else "Q.D. = คำนวณไม่ได้")
            print(f"ค่าเฉลี่ย (Mean) = {result['mean']:.3f}")
            print(f"ค่าความแปรปรวน (Variance) = {result['variance']:.3f}")
            print(f"ค่าความเบี่ยงเบนมาตรฐาน (Std Dev) = {result['sd']:.3f}")
            print(f"ค่ามัธยฐาน (Median) = {result['Md']:.3f}" if result['Md'] is not None else "Median = คำนวณไม่ได้")
            print(f"ค่าฐานนิยม (Mode) = {result['Mo']:.3f}" if result["Mo"] is not None else "Mode = คำนวณไม่ได้")

        else:
            print("เลือกไม่ถูกต้อง กรุณาลองใหม่")

        print()
        again = input("ต้องการคำนวณอีกหรือไม่ (y/n): ").lower()
        if again != "y":
            clear_screen()
            print("ขอบคุณที่ใช้โปรแกรม")
            break


if __name__ == "__main__":
    main()

=== โปรแกรมคำนวณค่าสถิติ ===

1. ข้อมูลดิบ (ไม่แจกแจงความถี่)
2. ข้อมูลแจกแจงความถี่
0. ออกจากโปรแกรม

เลือกโหมด (0, 1 หรือ 2): 2

ป้อนข้อมูล (1-999) พิมพ์ 0 เพื่อหยุด:
(1)> 10
(2)> 40
(3)> 10
(4)> 60
(5)> 20
(6)> 70
(7)> 20
(8)> 65
(9)> 0
ป้อนจำนวนชั้น (ต้องอยู่ระหว่าง 3 - 15): 5

=== ตารางแจกแจงความถี่ ===
ช่วงคะแนน | Lo | Ho | จุดกลาง(x) | f | F | fx | สัดส่วน | ร้อยละ
---------------------------------------------------------------------------
 10-22  |  10 |  22 |    16.00 |   4 |   4 |   64.00 |   0.500 |  50.00
 23-35  |  23 |  35 |    29.00 |   0 |   4 |    0.00 |   0.000 |   0.00
 36-48  |  36 |  48 |    42.00 |   1 |   5 |   42.00 |   0.125 |  12.50
 49-61  |  49 |  61 |    55.00 |   1 |   6 |   55.00 |   0.125 |  12.50
 62-74  |  62 |  74 |    68.00 |   2 |   8 |  136.00 |   0.250 |  25.00
---------------------------------------------------------------------------
รวมทั้งหมด: 8 | fx รวม = 297.00

--- ค่าสถิติ  ---
ค่าความเบี่ยงเบนควอไทล์ (Q.D.) = 22.750
ค่าเฉลี่ย (Mean) = 37.